In [16]:
import requests
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd
import pprint
import re

# 관광지 목록 가져오기
FILE_PATH = "관광지_분류위도경도_0122_2-1.csv"
location_data = pd.read_csv(FILE_PATH, 
                       names=["분류", "장소명", "카카오맵_장소명", "검색_장소명", "위도", "경도", "주소"], 
                       encoding="euc-kr", header=0)

# 게시글 가져올 기간 설정(숫자형으로)
SEARCH_START_DATE = 20190101
SEARCH_END_DATE = 20190530

# 인스타그램 검색이 안되는 관광지명 저장
no_result_insta = []

# 관광지명으로 인스타그램 크롤링
for i in range(len(location_data)):
    print("***현재까지 진행완료", i, " / ", len(location_data))
    
    post_info_list = [] # 관광지 1곳의 각 게시글에 대한 정보 배열 초기화
    location = location_data.iloc[i]
    category_name = location["분류"]
    
    # 관광지명으로 해시태그 검색을 위한 URL 설정
    keyword = location["검색_장소명"].replace(" ", "") # 공백제거
    keyword_for_url = urllib.parse.quote(keyword)
    tag_search_url = "https://www.instagram.com/explore/tags/{}/".format(keyword_for_url)

    # chrome driver load 후 인스타그램 검색
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(tag_search_url)
    sleep(5)

    # 검색결과 나올 때 로그인 요청 DIV가 뜨는 경우가 있어서 예외처리
    try:
        require_login_close_button = driver.find_element_by_xpath("//button[@class='dCJp8 afkep xqRnw']")
        print("로그인버튼 클릭 전->", category_name, keyword)
        require_login_close_button.click()
        print("로그인버튼 클릭 후")
    except:
        print("Exception : 로그인창 없음->", category_name, keyword)
        
    sleep(2)

    # 상세게시물을 보기 위한 '최근 사진'영역에서 첫번째 게시물(제일 최근 게시물) 클릭 event
    try:
        print("최근사진 클릭 전->", category_name, keyword)
        driver.find_element_by_xpath("//h2[text()='최근 사진']/following-sibling::div/div/div[1]/div[1]").click()
        print("최근사진 클릭 후")
        sleep(5)
    except:
        print("Exception : 인스타 검색결과 없음->", category_name, keyword)
        no_result_insta.append((category_name, keyword))
        continue

    search_result_page_source = driver.page_source
    bsObj = BeautifulSoup(search_result_page_source, "lxml")

    # 각 게시글 상세보기 화면으로 이동하면서 정보 가져오기
    while True:
        sleep(3)
        
        # 게시글 상세보기 화면 open
        try:
            detail_page_source = driver.page_source
            bsObj = BeautifulSoup(detail_page_source, "lxml")
            
            post_id = bsObj.find("a", {"class" : "FPmhX notranslate nJAzx"}).text # 게시자ID
            post_date = bsObj.find_all("time")[0].attrs["datetime"][:10].replace("-", "") # 게시일자(YYYYMMDD)

            # 게시일자가 '검색시작일보다 이전 날짜'이면 loop 종료(최근게시글순이므로 더 검색할 필요없음)
            if int(post_date) < SEARCH_START_DATE:
                break

            # 게시일자가 '검색종료일보다 이전 날짜'이면 배열에 추가
            if int(post_date) <= SEARCH_END_DATE:
                post_info_list.append((category_name, keyword, post_date, post_id))
            
        except:
            print("Exception : 게시글(post)이 없거나 오류 발생!->", category_name, keyword)
            
            if len(post_info_list) <= 0:
                no_result_insta.append((category_name, keyword))
                
            driver.close()
            sleep(3)
            break
            
        # 다음 게시글로 클릭 event
        try:
            WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.HBoOv.coreSpriteRightPaginationArrow')))
            driver.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow').click()
        except:
            print("Exception : 다음 게시글(post)이 없거나 오류 발생!->", category_name, keyword)
            
            if len(post_info_list) <= 0:
                no_result_insta.append((category_name, keyword))
            
            driver.close()
            sleep(3)
            break
    
    # 관광지 1개의 DataFrame 생성(게시일자, 게시자ID)
    if len(post_info_list) > 0:
        df = pd.DataFrame(post_info_list, columns=["분류", "검색_장소명", "게시일자", "게시자ID"])
        df = df.drop_duplicates().groupby(["분류", "검색_장소명", "게시일자"]).count()

        print()
        print(df)
        print()

        # csv파일로 내보내기
        if len(df) > 0:
            df.to_csv("instagram_result.csv", 
                      mode="a",
                      header=False,
                      index=True,
                      encoding="euc-kr")
    
    # 게시글 정보를 하나도 배열에 담지 않으면 해당 해시태그(keyword)를 no_result 데이터에 추가
    else:
        no_result_insta.append((category_name, keyword))
    
    # 창닫기
    try:
        driver.close()
    except:
        print("Exception : 창이 이미 닫혀있음->", category_name, keyword)

# 인스타그램 검색결과 없는 관광지 별도 파일로 저장
if len(no_result_insta) > 0:
    no_result_df = pd.DataFrame(no_result_insta, columns=["분류", "검색_장소명"])
    no_result_df = no_result_df.drop_duplicates()

    # csv파일로 내보내기
    no_result_df.to_csv("instagram_no_result.csv", 
                          mode="a",
                          header=True,
                          index=False,
                          encoding="euc-kr")

# 창닫기
try:
    driver.close()
except:
    print("Exception : 창이 이미 닫혀있음")

로그인버튼 클릭 전-> 자연 1100고지습지
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 1100고지습지
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 1100고지습지
Exception : 창이 이미 닫혀있음-> 자연 1100고지습지
로그인버튼 클릭 전-> 자연 가마오름
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 가마오름
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 가마오름
Exception : 창이 이미 닫혀있음-> 자연 가마오름
로그인버튼 클릭 전-> 자연 가메창
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 가메창
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 가메창
Exception : 창이 이미 닫혀있음-> 자연 가메창
로그인버튼 클릭 전-> 자연 가세오름
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 가세오름
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 가세오름
Exception : 창이 이미 닫혀있음-> 자연 가세오름
로그인버튼 클릭 전-> 자연 가시오름
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 가시오름
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 가시오름
Exception : 창이 이미 닫혀있음-> 자연 가시오름
로그인버튼 클릭 전-> 자연 가파도
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 가파도
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 가파도
Exception : 창이 이미 닫혀있음-> 자연 가파도
로그인버튼 클릭 전-> 자연 갈마못
로그인버튼 클릭 후
최근사진 클릭 전-> 자연 갈마못
최근사진 클릭 후
Exception : 게시글(post)이 없거나 오류 발생!-> 자연 갈마못
Exception : 창이 이미 닫혀있음-> 자연 갈마못
로그인버튼 클릭 전-> 자연 감낭오름
로그

KeyboardInterrupt: 

In [10]:
import pandas as pd

test_list = [("체험", "도자기마을", "20190101", "aaa"), 
             ("체험", "도자기마을", "20190101", "bbbb"), 
             ("체험", "도자기마을", "20190102", "aaa1"),
            ("체험", "도자기마을", "20190101", "aaa1"), 
             ("체험", "도자기마을", "20190102", "aaa2"), 
             ("체험", "도자기마을", "20190103", "asd"),
            ("체험", "도자기마을", "20190103", "ccccc"), 
             ("체험", "도자기마을", "20190101", "jjj"), 
             ("체험", "도자기마을", "20190103", "ccccc")]

df = pd.DataFrame(test_list, columns=["분류", "관광지명", "게시일", "게시자ID"])
df = df.drop_duplicates().groupby(["분류", "관광지명", "게시일"]).count()

df.to_csv("test_result.csv", 
              mode="a", 
              header=True, 
              index=True,
              encoding="euc-kr")

df

,분류,관광지명,게시일,게시자ID
0,체험,도자기마을,20190101,aaa
1,체험,도자기마을,20190101,bbbb
2,체험,도자기마을,20190102,aaa1
3,체험,도자기마을,20190101,aaa1
4,체험,도자기마을,20190102,aaa2
5,체험,도자기마을,20190103,asd
6,체험,도자기마을,20190103,ccccc
7,체험,도자기마을,20190101,jjj
8,체험,도자기마을1,20190103,ccccc
